In [1]:
# Dependencies

import pandas as pd
import requests
import json
import pprint
import numpy as np
import matplotlib.pyplot as plt
import requests
from census import Census
import gmaps
from us import states

# Census & gmaps API Keys
from config import (api_key, gkey)
year = 2012
c = Census(api_key, year)

# Configure gmaps
gmaps.configure(api_key=gkey)

In [2]:
datalst = []

In [3]:
for year in range(2012, 2018):
    c = Census(api_key, year) 

    data = c.acs5.get(('NAME', "B25077_001E","B25064_001E", "B15003_022E","B19013_001E"), 
                      {'for': 'zip code tabulation area:*'}) 

    cleandata = (pd.DataFrame(data) 
                   .rename(columns={"NAME": "Name", 
                                    "zip code tabulation area": "Zipcode", 
                                    "B25077_001E": "Median_Home_Value", 
                                    "B25064_001E": "Median_Rent", 
                                    "B15003_022E": "Bachelor_Degrees", 
                                    "B19013_001E": "Median_Income"}) 
                   .astype({'Zipcode':'int64'}) 
                   .query('(Median_Home_Value > 0) & (Median_Rent > 0) & (Median_Income > 0)')
                   .dropna()
                   .assign(Year = year)
                 )

    datalst.append(cleandata)


final_data = pd.concat(datalst, ignore_index = True)

In [4]:
final_data.head()


,Name,Median_Home_Value,Median_Rent,Bachelor_Degrees,Median_Income,Zipcode,Year
0,ZCTA5 02322,310500.0,1131.0,456.0,70481.0,2322,2012
1,ZCTA5 02324,333900.0,1144.0,3856.0,90722.0,2324,2012
2,ZCTA5 02330,269100.0,1547.0,1178.0,67963.0,2330,2012
3,ZCTA5 02332,573900.0,957.0,3949.0,117197.0,2332,2012
4,ZCTA5 02333,325700.0,993.0,1430.0,80169.0,2333,2012


In [5]:
#Seperate DF By Year
data2012 = final_data.query('(Year == 2012)')
data2013 =final_data.query('(Year == 2013)')
data2014 =final_data.query('(Year == 2014)')
data2015 =final_data.query('(Year == 2015)')
data2016 =final_data.query('(Year == 2016)')
data2017 =final_data.query('(Year == 2017)')

In [6]:
#Change DF
mergerdf = data2012.merge(data2017,how='outer',on='Name',suffixes=('_2012','_2017'))

In [7]:
mergedf = mergerdf.dropna()

In [8]:
mergedf.head()

,Name,Median_Home_Value_2012,Median_Rent_2012,Bachelor_Degrees_2012,Median_Income_2012,Zipcode_2012,Year_2012,Median_Home_Value_2017,Median_Rent_2017,Bachelor_Degrees_2017,Median_Income_2017,Zipcode_2017,Year_2017
0,ZCTA5 02322,310500.0,1131.0,456.0,70481.0,2322.0,2012.0,294900.0,1116.0,697.0,69709.0,2322.0,2017.0
1,ZCTA5 02324,333900.0,1144.0,3856.0,90722.0,2324.0,2012.0,338000.0,1385.0,3700.0,88640.0,2324.0,2017.0
2,ZCTA5 02330,269100.0,1547.0,1178.0,67963.0,2330.0,2012.0,274600.0,1319.0,1066.0,73904.0,2330.0,2017.0
3,ZCTA5 02332,573900.0,957.0,3949.0,117197.0,2332.0,2012.0,594000.0,2176.0,4124.0,123613.0,2332.0,2017.0
4,ZCTA5 02333,325700.0,993.0,1430.0,80169.0,2333.0,2012.0,321600.0,1148.0,1691.0,86568.0,2333.0,2017.0


In [9]:
#Change in Home Value
mergedf['Change_In_Value']=(mergedf['Median_Home_Value_2017']-mergedf['Median_Home_Value_2012'])/(mergedf['Median_Home_Value_2012'])

C:\Users\Micha\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [10]:
#Change in Rent
mergedf['Change_In_Rent']=(mergedf['Median_Rent_2017']-mergedf['Median_Rent_2012'])/(mergedf['Median_Rent_2012'])

C:\Users\Micha\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [11]:
#Change in Degree
mergedf['Change_In_Degree']=(mergedf['Bachelor_Degrees_2017']-mergedf['Bachelor_Degrees_2012'])/(mergedf['Bachelor_Degrees_2012'])

C:\Users\Micha\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [12]:
mergedf.head()

,Name,Median_Home_Value_2012,Median_Rent_2012,Bachelor_Degrees_2012,Median_Income_2012,Zipcode_2012,Year_2012,Median_Home_Value_2017,Median_Rent_2017,Bachelor_Degrees_2017,Median_Income_2017,Zipcode_2017,Year_2017,Change_In_Value,Change_In_Rent,Change_In_Degree
0,ZCTA5 02322,310500.0,1131.0,456.0,70481.0,2322.0,2012.0,294900.0,1116.0,697.0,69709.0,2322.0,2017.0,-0.050242,-0.013263,0.528509
1,ZCTA5 02324,333900.0,1144.0,3856.0,90722.0,2324.0,2012.0,338000.0,1385.0,3700.0,88640.0,2324.0,2017.0,0.012279,0.210664,-0.040456
2,ZCTA5 02330,269100.0,1547.0,1178.0,67963.0,2330.0,2012.0,274600.0,1319.0,1066.0,73904.0,2330.0,2017.0,0.020438,-0.147382,-0.095076
3,ZCTA5 02332,573900.0,957.0,3949.0,117197.0,2332.0,2012.0,594000.0,2176.0,4124.0,123613.0,2332.0,2017.0,0.035024,1.273772,0.044315
4,ZCTA5 02333,325700.0,993.0,1430.0,80169.0,2333.0,2012.0,321600.0,1148.0,1691.0,86568.0,2333.0,2017.0,-0.012588,0.156093,0.182517


In [13]:
#Change in Percent Data Frame
changedf = mergedf[['Name','Change_In_Value','Change_In_Rent','Change_In_Degree','Zipcode_2012']]

In [14]:
changedf = changedf.rename(columns = {'Zipcode_2012':'Zipcode'}).astype({'Zipcode':'int64'})

In [25]:
####formatteddf = changedf.style.format({'Change_In_Value': "{:.2%}",'Change_In_Rent': "{:.2%}",'Change_In_Degree': "{:.2%}"})

AttributeError: 'DataFrame' object has no attribute 'format'

In [23]:
changedf.head()

,Name,Change_In_Value,Change_In_Rent,Change_In_Degree,Zipcode
0,ZCTA5 02322,-0.050242,-0.013263,0.528509,2322
1,ZCTA5 02324,0.012279,0.210664,-0.040456,2324
2,ZCTA5 02330,0.020438,-0.147382,-0.095076,2330
3,ZCTA5 02332,0.035024,1.273772,0.044315,2332
4,ZCTA5 02333,-0.012588,0.156093,0.182517,2333
